In [1]:
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
from tqdm import tqdm
import pickle
import copy
from sklearn.model_selection import train_test_split
from models.pytorch_model import OrigamiNetwork
from vis_helpers import *

In [2]:
def layer_sam(D, n):
    scales = (D@n)/np.dot(n, n)
    mask = scales > 1
    return D + mask[:,np.newaxis] * (2*n - (2 * np.outer(scales, n)))
fv1 = np.array([0,.5])
size = 10
X1, X2 = np.meshgrid(np.linspace(-2,2,size), np.linspace(-2,2,size))
sandwichX = np.concatenate((X1.reshape(-1,1), X2.reshape(-1,1)), axis=1)
sandwichY = layer_sam(sandwichX, fv1)[:,1] > -.5
# plt.scatter(sandwichX[:,0], sandwichX[:,1], c=sandwichY)

size = 100
x1 = np.linspace(-2,2,size)
x2 = np.linspace(-2,2,size)
X1,X2 = np.meshgrid(x1,x2)
xorX = np.concatenate((X1.reshape(-1,1),X2.reshape(-1,1)),axis=1)
xorY = np.array([[1 if (i < size//2 and j < size//2 or i >= size//2 and j >= size//2) else 0 for j in range(size)] for i in range(size)]).reshape(-1)
# plt.scatter(xorX[:,0], xorX[:,1], c=xorY)

count = 20
onedX = np.linspace(-2,2,count)
onedY = np.zeros(count)
onedY[:6] = 1
onedY[-6:] = 1
onedX = np.expand_dims(onedX, axis=1)

In [3]:
test_case_x = sandwichX
test_case_y = sandwichY
print("Testing XOR problem")

test = OrigamiNetwork(n_layers=1, width=None, epochs=500, learning_rate=0.0001, optimizer_type="grad", reg=0, leak=0.08)
test.output_layer = np.array([[-0.96438805, 0.36995705],
                                [-1.13132166 ,-1.41350615]])
test.b = np.array([0.95990296, 0.72748519])
test.fold_vectors = np.array([[1., 0.25],])
                            #   [0.01, 0.01]])
histories = test.fit(test_case_x, test_case_y, freeze_cut=True, freeze_folds=False)
out = test.forward_pass(test_case_x)

print("\nScore =", test.score(), "after training")
print("Cut", test.output_layer, test.b, "Fold vectors", test.fold_vectors, sep="\n")

Testing XOR problem


TypeError: OrigamiNetwork.__init__() got an unexpected keyword argument 'layers'

In [ ]:
plot_wiggles(layer=0, histories)

In [ ]:
plot_history(test, verbose=2)

In [ ]:
max_score, best_folds = test.iscore_landscape(score_layers=None, create_plot=True, density=31, verbose=1, learning=False)
print(test.output_layer)